# <a name="dispatcher-bohrium"></a> Use DPDispatcher executor to submit jobs to Bohrium

Define a Python OP

In [1]:
from pathlib import Path
from dflow.python import OP, Artifact, PythonOPTemplate

@OP.function
def duplicate(num: int, foo: Artifact(Path)) -> {"num": int, "bar": Artifact(Path)}:
    with open(foo, "r") as f:
        content = f.read()
    with open("bar.txt", "w") as f:
        f.write(content * 2)
    return {
        "num": num * 2,
        "bar": Path("bar.txt"),
    }

Define a dispatcher executor

In [2]:
from dflow.plugins.dispatcher import DispatcherExecutor

dispatcher_executor = DispatcherExecutor(
        machine_dict={
            "batch_type": "Bohrium",
            "context_type": "Bohrium",
            "remote_profile": {
                "email": "<your-bohrium-email>",
                "password": "<your-bohrium-password>",
                "program_id": 2426,
                "input_data": {
                    "job_type": "container",
                    "platform": "ali",
                },
            },
        },
    )

Upload the input file

In [3]:
from dflow import upload_artifact

with open("foo.txt", "w") as f:
    f.write("Hello")
art = upload_artifact("foo.txt")

Instantiate a step with the dispatcher executor and the uploaded artifact

In [4]:
from dflow import Step

step = Step(
    name="step",
    template=PythonOPTemplate(duplicate, image="python:3.8"),
    parameters={"num": 3},
    artifacts={"foo": art},
    executor=dispatcher_executor,
)

Add step into a workflow

In [5]:
from dflow import Workflow

wf = Workflow(name="dispatcher-bohrium")
wf.add(step)

Submit the workflow

In [6]:
wf.submit()

Workflow has been submitted (ID: dispatcher-bohrium-5s5s8, UID: b6420324-f679-4f15-8029-c50464206f8e)


{'metadata': {'name': 'dispatcher-bohrium-5s5s8', 'generateName': 'dispatcher-bohrium-', 'namespace': 'argo', 'uid': 'b6420324-f679-4f15-8029-c50464206f8e', 'resourceVersion': '1948916', 'generation': 1, 'creationTimestamp': '2022-11-20T09:29:15Z', 'labels': {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2022-11-20T09:29:15Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}}, 'f:spec': {}, 'f:status': {}}}]}, 'spec': {'templates': [{'name': 'dispatcher-bohrium-steps', 'inputs': {}, 'outputs': {}, 'metadata': {}, 'steps': [[{'name': 'step', 'template': 'duplicate-i1yoq-ans39', 'arguments': {'parameters': [{'name': 'num', 'value': '3', 'description': '{"type": "<class \'int\'>"}'}], 'artifacts': [{'name': 'foo', 'path': '/tmp/inputs/artifacts/foo', 's3': {'key': 'u

# <a name="Argo-UI"></a> Argo UI

After finishing the previous steps, we can access the workflow we just ran on the UI (https://127.0.0.1:2746)